In [95]:
import xarray as xr
import matplotlib.pyplot as plt
import os
import csv

from datetime import datetime, timedelta
import io

from metpy.units import units
from metpy.plots import ImagePlot, MapPanel, PanelContainer
from siphon.catalog import TDSCatalog
import xarray as xr

import cartopy.crs

In [96]:
ds = xr.load_dataset("hrrr.t00z.wrfsfcf00_1.grib2", engine="cfgrib")

In [100]:
ds

<xarray.Dataset>
Dimensions:            (y: 1059, x: 1799)
Coordinates:
    time               datetime64[ns] 2021-11-07
    step               timedelta64[ns] 00:00:00
    heightAboveGround  float64 2.0
    latitude           (y, x) float64 21.14 21.15 21.15 ... 47.86 47.85 47.84
    longitude          (y, x) float64 237.3 237.3 237.3 ... 299.0 299.0 299.1
    valid_time         datetime64[ns] 2021-11-07
Dimensions without coordinates: y, x
Data variables:
    t2m                (y, x) float32 295.0 295.0 295.0 ... 281.7 281.7 281.7
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP 
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP 
    history:                 2021-12-07T14:42 GRIB to CDM+CF via cfgrib-0.9.9...

In [98]:
latitude = float(ds['latitude'][0][0]['latitude'])
longitude = float(ds['latitude'][0][0]['longitude'])
value = float(ds['t2m'][0][0])
coordinates = str(str(latitude)+', '+str(longitude))


cmd_tmp = "/usr/bin/wgrib2 %s -match '%s' -lon %s >> urma_temp.csv" % ('hrrr.t00z.wrfsfcf00_1.grib2', ':TMP:2 m above ground:anl:', coordinates)
    
os.system(cmd_tmp)
        
with open('urma_temp.csv', 'r') as f:
    csv_reader = csv.reader(f)
    csv_list = list(csv_reader)
        
current_value = float((csv_list[0][2]).split('=')[1])
print(current_value)

272.599


In [99]:
img = ImagePlot()
img.data = ds
img.field = 't2m'
img.colormap = 'coolwarm'

panel = MapPanel()
panel.area = 'us'
panel.layers = ['states']
panel.title = 'GFS precip forecast'
panel.plots = [img]

pc = PanelContainer()
pc.size = (10, 8)
pc.panels = [panel]
pc.show()

/home/hliu/Projects/modap/env/lib/python3.9/site-packages/metpy/xarray.py:349: UserWarning: More than one time coordinate present for variable "t2m".
  warnings.warn('More than one ' + axis + ' coordinate present for variable'


AttributeError: crs attribute is not available.

<Figure size 720x576 with 0 Axes>

In [ ]:
for x in range(len(ds['latitude'])):
    for y in range(len(ds['latitude'][0])):
        latitude = float(ds['latitude'][x][y]['latitude'])
        longitude = float(ds['latitude'][x][y]['longitude'])
        value = float(ds.t2m[x][y])
        coordinates = str(str(latitude)+', '+str(longitude))
        
        cmd_tmp = "/usr/bin/wgrib2 %s -match '%s' -lon %s >> urma_temp.csv" % 
        ('GFS DATA HERE', ':TMP:2 m above ground:anl:', coordinates)
    
        os.system(cmd_tmp)
        
        with open('urma_temp.csv', 'r') as f:
          csv_reader = csv.reader(f)
          csv_list = list(csv_reader)
        
        gfs_value = float((csv_list[0][2]).split('=')[1])
        
        ds.t2m[x][y] = (value+gfs_value)/2